In [5]:
import re

import pandas as pd
from pyproj import Transformer

In [6]:
noise_data = pd.read_csv("../local_data/noise_data_raw.csv")

In [7]:
def extract_coordinates(point_str: str) -> tuple[float, float, float]:
    """
    Extract x, y, z coordinates from a POINT Z string.

    Args:
        point_str: String in format 'POINT Z (x y z)'

    Returns:
        Tuple containing x, y, z coordinates as floats
    """
    # Extract numbers using regex
    coords = re.findall(r"[-+]?\d*\.\d+|\d+", point_str)
    return float(coords[0]), float(coords[1]), float(coords[2])


# Create transformer from CH1903+/LV95 to WGS84
transformer = Transformer.from_crs("EPSG:2056", "EPSG:4326")

# Extract coordinates and transform them
coords_list = noise_data["geometry"].apply(extract_coordinates)
transformed_coords = coords_list.apply(lambda x: transformer.transform(x[0], x[1]) + (x[2],))

# Add new columns
noise_data["latitude"] = [coord[0] for coord in transformed_coords]
noise_data["longitude"] = [coord[1] for coord in transformed_coords]
noise_data["elevation"] = [coord[2] for coord in transformed_coords]

In [8]:
noise_data.drop(columns=["geometry"]).to_csv("../local_data/noise_data_cleaned.csv", index=False)